In [1]:
import requests
from bs4 import BeautifulSoup
import json
import re
import random ##kann raus
from datetime import datetime
from collections import OrderedDict


def extract_pattern(string, pattern):
    match = re.search(pattern, string)
    if match:
        return match.group(1)
    else:
        return None

def format_timestring(timestring):
    timestring = timestring.replace('a.m.','AM').replace('p.m.','PM')
    b= timestring.split()
    if len(b[0])>3:
            b[0] = b[0][:3]+'.'
    if len(b[3]) not in [4,5]:
            b[3] = '0'+b[3]+':00'
    b=' '.join(b)
    return b
initial_uid='424df273-9a73-49a4-b63a-38fc50387f15'



### Fetch Results

In [2]:
###NLST

url_leaderboard = 'https://learn2reg.grand-challenge.org/evaluation/nlst-validation/leaderboard/' ##NLST 
max_submissions = 1000

#submissions = {}
#ordered dict
submissions = OrderedDict()

##regex patterns
username_pattern = r'<a\s+href="[^"]+\/users\/([^\/]+)\/"[^>]*>[^<]+<\/a>'
team_pattern = r'\(<a\s+href="[^"]+">(.*?)<\/a>\)'
uid_pattern = r'\/evaluation\/([\w-]+)\/'

url_suffix = '?draw=1&columns[0][data]=0&columns[0][name]=&columns[0][searchable]=true&columns[0][orderable]=true&columns[0][search][value]=&columns[0][search][regex]=false&order[0][column]=0&order[0][dir]=asc&start=0&length={}&search[value]=&search[regex]=false&_=1686904037262'.format(max_submissions)
response = requests.get(url_leaderboard+url_suffix, headers={'X-Requested-With': 'XMLHttpRequest'})
num_records = int(response.json()['recordsTotal'])
table_data = response.json()['data']
print('Found {} entries'.format(num_records))

for ii,examplary_entry in enumerate(range(len(table_data))):
        #probably better with regex
        #submission_uid = table_data[examplary_entry][3].split()[1].split('/')[4]
        submission_uid = extract_pattern(table_data[examplary_entry][3], uid_pattern)
        submission_user = extract_pattern(table_data[examplary_entry][1], username_pattern)
        submission_team = extract_pattern(table_data[examplary_entry][1], team_pattern)
        submissions[submission_uid] = {'user': submission_user, 'team': submission_team}

        ##get json directly
        url_submission = 'https://learn2reg.grand-challenge.org/evaluation/'+submission_uid+'/'
        response_submission = requests.get(url_submission)
        soup = BeautifulSoup(response_submission.text, 'html.parser')
        results = json.loads(str(soup.findAll('pre')[0])[5:-6])
        #time
        time_string = soup.findAll('dt')[7].next_sibling.next_sibling.text
        time = datetime.strptime(format_timestring(time_string), '%b. %d, %Y, %I:%M %p')
        
        submissions[submission_uid] = {'user': submission_user, 'team': submission_team, 'table_rank':str(ii+1), 'time':str(time), 'results': results}
        print(ii,submission_uid, submission_user, submission_team, time, sep='\t')
        
print('Fetched {} entries'.format(len(submissions)))

##save dict as sorted json
#with open('L2R2023_Snapshot_NLST.json', 'w') as fp:
#    json.dump(submissions, fp, indent=4, sort_keys=False)
    


Found 84 entries
0	17aa585f-2ce8-46c4-8876-b472daaa5028	dericdesta	VROC	2023-08-04 10:02:00
1	c2ab9bf6-19b4-43f4-b9f2-2b590ac41397	TS_UKE	VROC	2023-08-04 21:06:00
2	65939254-621a-484e-81ee-4883cb05d8f0	cwmokab	None	2023-08-07 17:11:00
3	83e61e15-582d-4cd0-a15c-80920033fa4b	cwmokab	None	2023-07-07 12:53:00
4	fb620ad1-b534-4976-8e7e-2f30ad8866d9	xi	None	2023-07-18 12:26:00
5	51a14adc-8e32-4371-9816-8a933988a746	Birmingham	None	2023-07-07 12:57:00
6	32db63be-61df-410f-97fc-4d87253f7c98	yihaol	None	2023-07-07 13:06:00
7	0aae1073-904c-4de9-8353-0a650abc20c1	cwmokab	None	2023-08-07 18:39:00
8	39be3e72-9dec-4957-a3a7-2b9872af2160	xi	None	2023-07-07 13:00:00
9	df75259b-124b-46d8-80e6-cc7c2ec192c8	alisonbrielee	None	2023-08-07 12:49:00
10	81ac61aa-3098-451b-b20e-42aa89497cf2	xi	None	2023-07-19 09:13:00
11	6f6d3fc2-9fce-493b-b37d-91f7ac1f5fd3	pablomiguelgarcia1990	None	2023-08-03 09:12:00
12	aa6b77e1-9545-4c75-9242-5ff75a283130	cwmokab	None	2023-08-02 06:22:00
13	6dcba64e-f890-43d2-979d-876d7763

In [3]:

beginningSubmission = datetime.strptime('2023-07-08','%Y-%m-%d')
endSubmission = datetime.strptime('2023-08-06 23:59:59','%Y-%m-%d %H:%M:%S')


#with open('L2R2023_Snapshot_NLST.json') as json_file:
#    submissions = json.load(json_file)
#
#results = OrderedDict()
#for submission in results_json:
#    submission_date = datetime.strptime(results_json[submission]['time'],'%Y-%m-%d %H:%M:%S')
#    if submission_date > beginningSubmission and submission_date < endSubmission:
#        results[submission] = results_json[submission]


##add initial as well
#ThoraxCBCT:     2c0360d0-ee97-4b37-a950-9ad1edb1ed5b
#initial_uid = '2c0360d0-ee97-4b37-a950-9ad1edb1ed5b'





##Remove submissions that are outside of the submission window
results = OrderedDict()
for submission in submissions:
    submission_date = datetime.strptime(submissions[submission]['time'],'%Y-%m-%d %H:%M:%S')
    if submission_date > beginningSubmission and submission_date < endSubmission:
        results[submission] = submissions[submission]
print('Before:' + str(len(submissions)), 'After:' + str(len(results)))

results[initial_uid] = submissions[initial_uid]
results[initial_uid]['user']='INITIAL'
results[initial_uid]['team']=None






Before:84 After:58


### Calculate Ranks

In [4]:
import numpy as np
import scipy.stats as stats
import scipy 
def unique(list1):
    # insert the list to the set
    list_set = set(list1)
    # convert the set to the list
    unique_list = (list(list_set))
    return unique_list
np.set_printoptions(precision=3)


In [5]:
sample_submission = results[list(results.keys())[0]]
all_cases = list(sample_submission['results']['cases'])


#######
uids = list(results.keys())
users = [results[k]['user'] for k in results.keys()]
teams = [results[k]['team'] for k in results.keys()]
table_rank = [results[k]['table_rank'] for k in results.keys()]
results_q = [results[k]['results'] for k in results.keys()]



TRE_kp = np.empty((len(users), 21349))
TRE_lm = np.empty((len(users), 1000))

LogJacDetStd = np.ones((len(users),len(all_cases)))

for i, uid in enumerate(uids):
    tmp_kp = []
    tmp_lm = []
    for j, case in enumerate(all_cases):
        LogJacDetStd[i,j] = results_q[i]['cases'][case]['LogJacDetStd']['detailed']
        tmp_kp += results_q[i]['cases'][case]['TRE_kp']['detailed']
        tmp_lm += results_q[i]['cases'][case]['TRE_lm']['detailed']
    TRE_kp[i] = tmp_kp
    TRE_lm[i] = tmp_lm


N = len(uids)

##zero displacements
idx0 = uids.index(initial_uid)
LogJacDetStd[idx0,j] = 5.0 ##set large value for initial 


###robistyfy

TRE_lm_30 = np.zeros((N, round((1000)*.3)))

idx30 = np.argsort(TRE_lm[idx0].reshape(-1))[:TRE_lm_30.shape[1]]
for i in range(N):
    TRE_lm_30[i] = TRE_lm[i].reshape(-1)[idx30]



##currently not used
TRE_kp_30 = np.zeros((N, round((21349)*.3)))
#print(TRE_kp.shape,TRE_kp_30.shape)
idx30 = np.argsort(TRE_kp[idx0].reshape(-1))[:TRE_kp_30.shape[1]]
for i in range(N):
    TRE_kp_30[i] = TRE_kp[i].reshape(-1)[idx30]








In [6]:

p_threshold = 0.05
def scores_better(task_metric):
    better = np.zeros((N,N))
    for i in range(N):
        for j in range(N):
            h,p = scipy.stats.ranksums(task_metric[i], task_metric[j])
            if((h>0)&(p<p_threshold)): #sign of h and p-value
                better[i,j] = 1
    scores_task = better.sum(0)
    return scores_task

def rankscore_avgtie(scores_int):
    N = len(scores_int)
    rankscale = np.linspace(.1,1,N) #our definition
    rankavg = np.zeros((N,2))
    scorerank = np.zeros(N)
    #argsort with reverse index
    idx_ = np.argsort(scores_int)
    idx = np.zeros(N).astype('int32')
    idx[idx_] = np.arange(N)
    #averaging ties
    for i in range(N):
        rankavg[scores_int[i],0] += rankscale[idx[i]]
        rankavg[scores_int[i],1] += 1
    rankavg = rankavg[:,0]/np.maximum(rankavg[:,1],1e-6)
    for i in range(N):
        scorerank[i] = rankavg[scores_int[i]]
    return scorerank
scores = np.array([0, 2, 4, 3, 4, 0]).astype('int32')
scorerank = rankscore_avgtie(scores)

def greaters(scores):
    return np.sum(scores.reshape(1,-1)>scores.reshape(-1,1),0)


In [7]:
##  Ranking

rank_all = np.zeros((N,4))
subset = TRE_lm.reshape(N,-1)
scores = scores_better(-subset)
rank_tre = rankscore_avgtie(-scores.astype('int64'))
rank_all[:,0] = rank_tre

subset = TRE_lm_30.reshape(N,-1)
scores = scores_better(-subset)
rank_tre_30 = rankscore_avgtie(-scores.astype('int64'))
rank_all[:,1] = rank_tre_30

subset = TRE_kp.reshape(N,-1)
scores = scores_better(-subset)
rank_kpt = rankscore_avgtie(-scores.astype('int64'))
rank_all[:,2] = rank_kpt

subset = LogJacDetStd.reshape(N,-1)
scores = scores_better(-subset)
rank_jac3 = rankscore_avgtie(-scores.astype('int64'))
rank_all[:,3] = rank_jac3



In [8]:
LogJacDetStd_mean = LogJacDetStd.reshape(N,-1).mean(1)
TRE_kp_mean = TRE_kp.reshape(N,-1).mean(1)
TRE_30_mean = TRE_lm_30.reshape(N,-1).mean(1)
TRE_mean = TRE_lm.reshape(N,-1).mean(1)


In [9]:
rank3 = np.power(np.prod(rank_all[:,:4],axis=1),.25)
all_rank3 = np.power(np.prod(rank_all[:,:4],axis=1),1/4)
idx2 = np.argsort(-all_rank3)



In [10]:
idx2 = np.argsort(-all_rank3)
for i in range(N):
    team = users[idx2[i]].replace("0","O").zfill(25).replace("0", " ")
    #print(i+1,'\t',team,'\t','%0.3f'%all_rank[idx2[i]],'\t','%0.3f'%TRE_lm_mean[idx2[i]],'\t','%0.3f'%np.mean(LogJacDetStd,1)[idx2[i]],'\t','https://learn2reg.grand-challenge.org/evaluation/'+uids[idx2[i]]+'/')
    gc_team = teams[idx2[i]] if str(teams[idx2[i]]) != 'None' else ' '
    print(i+1,'\t',team,'\t',gc_team,3*'\t','%0.3f'%all_rank3[idx2[i]],'\t','%0.3f'%TRE_mean[idx2[i]],'\t','%0.3f'%LogJacDetStd_mean[idx2[i]],'\t','https://learn2reg.grand-challenge.org/evaluation/'+uids[idx2[i]]+'/')


1 	                   cwmokab 	   			 0.865 	 1.672 	 0.032 	 https://learn2reg.grand-challenge.org/evaluation/c3ca0458-4e59-4f8e-bdcf-1e2c36f6d428/
2 	                dericdesta 	 VROC 			 0.837 	 0.908 	 0.082 	 https://learn2reg.grand-challenge.org/evaluation/17aa585f-2ce8-46c4-8876-b472daaa5028/
3 	                   cwmokab 	   			 0.830 	 1.667 	 0.038 	 https://learn2reg.grand-challenge.org/evaluation/aa6b77e1-9545-4c75-9242-5ff75a283130/
4 	                  funatomi 	   			 0.780 	 43.577 	 1.232 	 https://learn2reg.grand-challenge.org/evaluation/ee244ff5-9bcf-48c9-890f-19b6af043bb8/
5 	             alisonbrielee 	   			 0.779 	 1.761 	 0.046 	 https://learn2reg.grand-challenge.org/evaluation/a1f6e765-b99c-4240-92e0-25f0067633b2/
6 	                  funatomi 	   			 0.778 	 1.841 	 0.019 	 https://learn2reg.grand-challenge.org/evaluation/c0c4044b-e84d-43bd-bb7d-28aa99ec66a7/
7 	                  funatomi 	   			 0.759 	 1.952 	 0.019 	 https://learn2reg.grand-challenge.org/ev

In [11]:
##Remove team and participant duplicates

tmp_teams=[]
tmp_users=[]
idx3 = idx2.tolist().copy()
for i in idx2:
    if (teams[i] in tmp_teams and teams[i] != None) or (users[i] in tmp_users):
        idx3.remove(i)
    else:
        tmp_teams.append(teams[i])
        tmp_users.append(users[i])


final_idx = np.array(idx3)

for ii,idx in enumerate(final_idx):
    team = users[idx].replace("0","O").zfill(25).replace("0", " ")
    gc_team = teams[idx] if str(teams[idx]) != 'None' else ' '
    print(ii+1,'\t',team,'\t',gc_team,3*'\t','%0.3f'%all_rank3[idx],'\t','%0.3f'%TRE_mean[idx],'\t','%0.3f'%LogJacDetStd_mean[idx],'\t','https://learn2reg.grand-challenge.org/evaluation/'+uids[idx]+'/')


1 	                   cwmokab 	   			 0.865 	 1.672 	 0.032 	 https://learn2reg.grand-challenge.org/evaluation/c3ca0458-4e59-4f8e-bdcf-1e2c36f6d428/
2 	                dericdesta 	 VROC 			 0.837 	 0.908 	 0.082 	 https://learn2reg.grand-challenge.org/evaluation/17aa585f-2ce8-46c4-8876-b472daaa5028/
3 	                  funatomi 	   			 0.780 	 43.577 	 1.232 	 https://learn2reg.grand-challenge.org/evaluation/ee244ff5-9bcf-48c9-890f-19b6af043bb8/
4 	             alisonbrielee 	   			 0.779 	 1.761 	 0.046 	 https://learn2reg.grand-challenge.org/evaluation/a1f6e765-b99c-4240-92e0-25f0067633b2/
5 	                        xi 	   			 0.736 	 1.653 	 0.049 	 https://learn2reg.grand-challenge.org/evaluation/81ac61aa-3098-451b-b20e-42aa89497cf2/
6 	                   SSKJLBW 	 ikun 			 0.732 	 1.856 	 0.036 	 https://learn2reg.grand-challenge.org/evaluation/6d7af73a-27f5-48e0-85d2-205d00355422/
7 	          i.d.kolenbrander 	   			 0.731 	 2.537 	 0.042 	 https://learn2reg.grand-challenge.org

In [12]:
##pretty print with pandas
import pandas as pd
##printoptions precision
pd.set_option('precision', 3)

df = pd.DataFrame(columns=['#rank', 'user', 'team', '#score', 'TRE_lm(mean)', 'LogJacDetStd(mean)'])
for ii,idx in enumerate(final_idx):
    team = users[idx].replace("0","O").zfill(25).replace("0", " ")
    gc_team = teams[idx] if str(teams[idx]) != 'None' else ' '
    df.loc[ii] = [ii+1, users[idx], gc_team, all_rank3[idx], TRE_mean[idx], LogJacDetStd_mean[idx]]
print(df)

   #rank                   user             team  #score  TRE_lm(mean)  \
0      1                cwmokab                    0.865         1.672   
1      2             dericdesta             VROC   0.837         0.908   
2      3               funatomi                    0.780        43.577   
3      4          alisonbrielee                    0.779         1.761   
4      5                     xi                    0.736         1.653   
5      6                SSKJLBW             ikun   0.732         1.856   
6      7       i.d.kolenbrander                    0.731         2.537   
7      8          yunzhengzhu19  UCLA_MII_HsuLab   0.682         2.019   
8      9  pablomiguelgarcia1990                    0.612         1.663   
9     10                Kheil-Z    Tilted Towers   0.536         2.102   
10    11                 smajjk                    0.485         2.316   
11    12                INITIAL                    0.393        10.223   

    LogJacDetStd(mean)  
0           